In [12]:
#1. rutin1 import module
import pandas as pd
import os
import sys

In [13]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [14]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "notebooks/postager_nlp-id/dataset_ekstraksi_r29_pos_sm.xlsx")
df = pd.read_excel(dataset_path)
df_pos = df['pos_sentence_list']

In [15]:
from singlerank import SingleRank

pos = {'NN', 'VP', 'NP'}
# 1. create a TextRank extractor.
extractor = SingleRank()

In [ ]:
import ast 

df_keyphrases = pd.DataFrame(columns=['key_1', 'val_1', 'key_2', 'val_2', 'key_3', 'val_3'])

for i in range(len(df_pos)):
    try:
        text = df_pos[i]
        print('Processing Text', i, end='..! ')
        text1 = ast.literal_eval(text)
        extractor.load_document(input=text1)
        keyphrase = extractor.candidate_weighting_ia()

        # Separate the key and value elements into separate lists
        keys = [item[0] for item in keyphrase]
        values = [item[1] for item in keyphrase]

        # Create a dictionary with the keys and values
        data = {
            'key_1': [keys[0]],
            'val_1': [values[0]],
            'key_2': [keys[1]],
            'val_2': [values[1]],
            'key_3': [keys[2]],
            'val_3': [values[2]],
        }
        df_keyphrase = pd.DataFrame(data)
        print('Done')
    except Exception as e:
        print('Error on text', i)
        data = {
            'key_1': ['na'],
            'val_1': [0],
            'key_2': ['na'],
            'val_2': [0],
            'key_3': ['na'],
            'val_3': [0],
        }
        df_keyphrase = pd.DataFrame(data)
    # Concatenate df_keyphrase with df_keyphrases
    df_keyphrases = pd.concat([df_keyphrases, df_keyphrase], ignore_index=True)


In [17]:
# Add df_keyphrases ke file excel dataset_ekstraksi_r30.xlsx
result_df = pd.concat([df, df_keyphrases], axis=1)

# simpan dalam dataset
#output_file = 'std_pke_textrank.xlsx'
#result_df.to_excel(output_file, index=False)

EVALUATION

In [18]:
from utils import eval, write_excel

targets = result_df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [19]:
# Evaluation TextRank
predict_textrank_list = result_df[['key_1','key_2','key_3']].values.tolist()
eval_textrank = eval(predict_textrank_list, targets, True).round(3)
eval_textrank.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank = eval_textrank[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_textrank.head(3)

,key_1,key_2,key_3,flex_recall,flex_prec
0,no_match,no_match,no_match,0.000,0.000
1,no_match,no_match,full_match,0.143,0.333
2,no_match,no_match,no_match,0.000,0.000


In [20]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall = eval_textrank['flex_recall'].mean()
textrank_prec = eval_textrank['flex_prec'].mean()
textrank_f1 = 2 * (textrank_prec * textrank_recall) / (textrank_prec + textrank_recall)

# Create a DataFrame with the scores
summary = pd.DataFrame({'textrank': [textrank_recall, textrank_prec, textrank_f1]}, index=['recall', 'precision', 'F1'])
summary = summary.round(3)
summary

,textrank
recall,0.100
precision,0.233
F1,0.140


In [21]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank = pd.concat([result_df, df_targets, eval_textrank], axis=1)

In [22]:
# Write predictions to excel file
sheet_name = 'pke_singlerank'
output_file = 'pke_singlerank.xlsx'
write_excel(predict_textrank, sheet_name, output_file)

/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:16: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Visualisasi